<a href="https://colab.research.google.com/github/Saki-I/seikabutsu_app/blob/main/Zenn_EdinetAPI_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

END_POINT = 'https://disclosure.edinet-fsa.go.jp/api/v1'
submission_info_endpoint = f'{END_POINT}/documents.json'
submission_request_parameters = {
    'date': '2022-01-18',
    'type': 2
}
submission_info_response = requests.get(
    submission_info_endpoint, submission_request_parameters
)
submission_info_json = submission_info_response.json()

In [ ]:
import pandas as pd

raw_submission_info_df = pd.DataFrame(submission_info_json['results'])
raw_submission_info_df.columns

Index(['seqNumber', 'docID', 'edinetCode', 'secCode', 'JCN', 'filerName',
       'fundCode', 'ordinanceCode', 'formCode', 'docTypeCode', 'periodStart',
       'periodEnd', 'submitDateTime', 'docDescription', 'issuerEdinetCode',
       'subjectEdinetCode', 'subsidiaryEdinetCode', 'currentReportReason',
       'parentDocID', 'opeDateTime', 'withdrawalStatus', 'docInfoEditStatus',
       'disclosureStatus', 'xbrlFlag', 'pdfFlag', 'attachDocFlag',
       'englishDocFlag', 'csvFlag', 'legalStatus'],
      dtype='object')

In [ ]:
# 重要なカラムに絞る
submission_info_df = raw_submission_info_df[['docID', 'edinetCode', 'secCode', 'filerName', 'docDescription']]
submission_info_df.head()

,docID,edinetCode,secCode,filerName,docDescription
0,S100N8K1,None,None,None,None
1,S100N5TR,E11518,None,三菱ＵＦＪ国際投信株式会社,半期報告書（内国投資信託受益証券）－第39期(令和3年4月20日－令和4年4月18日)
2,S100MXAV,E12444,None,三井住友トラスト・アセットマネジメント株式会社,有価証券報告書（内国投資信託受益証券）－第8期(令和2年10月17日－令和3年10月18日)
3,S100N8O7,E26446,60610,株式会社ユニバーサル園芸社,臨時報告書
4,S100MQ7D,E13591,None,新生インベストメント・マネジメント株式会社,有価証券報告書（内国投資信託受益証券）－第8期(令和3年4月17日－令和3年10月18日)


In [ ]:
# 有価証券報告書の情報を抽出する。
securities_report_infos = []
for i, row in submission_info_df.iterrows():
    doc_desc = row['docDescription']

    if doc_desc is None:
        continue

    if ('有価証券報告書' in doc_desc) and ('受益証券' not in doc_desc):
        row_to_dataframe = pd.DataFrame([row])
        securities_report_infos.append(row_to_dataframe)

if len(securities_report_infos) == 0:
    print('有価証券報告書の提出情報がありません。')
else:
    print(f'{len(securities_report_infos)} 件の有価証券報告書が抽出されました。')
    securities_report_info_df = pd.concat(securities_report_infos)

4 件の有価証券報告書が抽出されました。


In [ ]:
securities_report_info_df

,docID,edinetCode,secCode,filerName,docDescription
111,S100N8SX,E33390,39940,株式会社マネーフォワード,訂正有価証券報告書－第8期(平成30年12月1日－令和1年11月30日)
115,S100N8ST,E33390,39940,株式会社マネーフォワード,訂正有価証券報告書－第9期(令和1年12月1日－令和2年11月30日)
121,S100N8OL,E00471,29010,石垣食品株式会社,訂正有価証券報告書－第63期(平成31年4月1日－令和2年3月31日)
158,S100N8L9,E31019,39000,株式会社クラウドワークス,訂正有価証券報告書－第10期(令和2年10月1日－令和3年9月30日)


In [ ]:
# 今回はマネーフォーワードの第9期決算情報を対象とする。
docID = 'S100N8ST'
document_endpoint = f'{END_POINT}/documents/{docID}'
document_request_parameters = {
    'type': 1
}
document_response = requests.get(document_endpoint, document_request_parameters)

In [ ]:
# まず、帰ってきたデータを zip 形式で保存する。
zip_file_full_path = f'/content/{docID}.zip'
with open(zip_file_full_path, 'wb') as f:
    for chunk in document_response.iter_content(chunk_size=1024):
        f.write(chunk)

In [ ]:
# zip ファイルを解凍する
import zipfile
import os
os.makedirs(f'/content/{docID}', exist_ok=True)
with zipfile.ZipFile(zip_file_full_path) as zip_f:
    zip_f.extractall(f'/content/{docID}')

In [ ]:
# xbrl ファイルを発見する
from glob import glob
# PublicDoc 内に格納されている xbrl ファイルが分析対象となるファイルである。
xbrl_expression = f'/content/{docID}/**/PublicDoc/**/*.xbrl'
xbrl_paths = glob(xbrl_expression, recursive=True)
xbrl_paths

['/content/S100N8ST/XBRL/PublicDoc/jpcrp030000-asr-001_E33390-000_2020-11-30_02_2022-01-18.xbrl']

In [ ]:
!pip install edinet_xbrl

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 571.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for edinet_xbrl: filename=edinet_xbrl-0.2.0-py2.py3-none-any.whl size=10846 sha256=c814bc4714b319fe9523eb09c8165327de1fed0b855613f41784abaab6adc676
  Stored in directory: /root/.cache/pip/wheels/9f/e5/02/7a2fb1d397b2e5e9badd3ce824287e8469404491af0d346de3
  Created wheel for python-xbrl: filename=python_xbrl-1.1.1-py2.py3-none-any.whl size=10612 sha256=5088b4413751e3f59ceeb7a57241ca14b1bd213ae3f1365b60466d094bc6f39a
  Stored in directory: /root/.cache/pip/wheels/6e/24/e6/16c20c0ef5fe7b59c03c893f883aa41ae4fdf30a77d2fd13ad
  Created wheel for ordereddict: filename=ordereddict-1.1-py3-none-any.whl size=3509 sha256=f3f9d83c5aa264582fcd742cd10af045bffed8b012934a5b750c13e3ff573468
  Stored in dire

In [ ]:
from edinet_xbrl.edinet_xbrl_parser import EdinetXbrlParser
parser = EdinetXbrlParser()
# Step2で特定した XBRL ファイルのパスを選択
xbrl_path = xbrl_paths[0]
parsed_xbrl = parser.parse_file(xbrl_path)

/usr/local/lib/python3.10/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [ ]:
# この決算情報がいつのものか確かめる。
key = "jpdei_cor:CurrentFiscalYearEndDateDEI"
context_ref = "FilingDateInstant"
report_end_date_info = parsed_xbrl.get_data_by_context_ref(key, context_ref)
report_end_date = report_end_date_info.get_value()
report_end_date

'2020-11-30'

In [ ]:
# 「事業の内容」という事業の概要について記述された情報を取得する。
key = "jpcrp_cor:DescriptionOfBusinessTextBlock"
context_ref = "FilingDateInstant"
summary_of_business_info = parsed_xbrl.get_data_by_context_ref(key, context_ref)
summary_of_business = summary_of_business_info.get_value()

In [ ]:
# 結果の出力
from IPython.display import HTML
HTML(summary_of_business)

In [ ]:
# 売上情報の取得
key = 'jpcrp_cor:NetSalesSummaryOfBusinessResults'
context_ref = 'CurrentYearDuration'
extracted_data = parsed_xbrl.get_data_by_context_ref(key, context_ref)
net_sales = extracted_data.get_value()
net_sales

'11318217000'

In [ ]:
# 有価証券報告書には基本5年分の情報があるため、5年分の情報をとってくる。
from typing import Dict, List
from datetime import datetime
from dateutil.relativedelta import relativedelta

durations = [
    'CurrentYearDuration',
    'Prior1YearDuration',
    'Prior2YearDuration',
    'Prior3YearDuration',
    'Prior4YearDuration'
]

def translate_period(report_end_date: str, duration: str) -> str:
    '''duration を対応する日付情報に変換'''
    report_end_date_datetime = datetime.strptime(report_end_date, '%Y-%m-%d')
    if duration == 'CurrentYearDuration':
        return report_end_date_datetime.strftime("%Y-%m-%d")
    else:
        n_year = int(duration[5])
        n_year_previous_date = report_end_date_datetime - relativedelta(years=n_year)
        return n_year_previous_date.strftime("%Y-%m-%d")

def get_all_periods_value(
    xbrl: EdinetXbrlParser,
    report_end_date: str,
    key: str,
    context_refs: List[str]
) -> Dict[str, str]:
    '''キーで指定した項目の各期間の情報を一括取得'''
    results: Dict[str, str] = {}
    for context_ref in context_refs:
        extracted_data = xbrl.get_data_by_context_ref(key, context_ref)
        _date = translate_period(report_end_date, context_ref)
        results[_date] = extracted_data.get_value()
    return results

In [ ]:
# 直近5年分の売上をまとめて取得する
net_sales = get_all_periods_value(
    xbrl=parsed_xbrl,
    key='jpcrp_cor:NetSalesSummaryOfBusinessResults',
    report_end_date=report_end_date,
    context_refs=durations
)
net_sales

{'2020-11-30': '11318217000',
 '2019-11-30': '7156784000',
 '2018-11-30': '4594789000',
 '2017-11-30': '2899548000',
 '2016-11-30': None}

In [ ]:
# 直近5年分の利益をまとめて取得する
net_profits = get_all_periods_value(
    xbrl=parsed_xbrl,
    key='jpcrp_cor:ProfitLossAttributableToOwnersOfParentSummaryOfBusinessResults',
    report_end_date=report_end_date,
    context_refs=durations
)
net_profits

{'2020-11-30': '-2423282000',
 '2019-11-30': '-2572050000',
 '2018-11-30': '-815445000',
 '2017-11-30': '-842814000',
 '2016-11-30': None}

In [ ]:
# @title
pip install Flask
